In [1]:
pip install -U keras-tuner

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=2c67829443261e24f738da8f6ec94bb1ce53c6aeba4db28ccf046d73a7d60a9f
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=973f1907a175dbc25764865d76f825840f3c07bb5d2f91da08c2c6f7a9f57adc
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
cd drive/My\ Drive/Projects/TimeSeriesForecasting

/content/drive/My Drive/Projects/TimeSeriesForecasting


In [0]:
import os
import json
import numpy as np
import pandas as po
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

In [4]:
'''
from keras import optimizers
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Input, Conv1D, Dense, Activation, Dropout, Flatten, Multiply, Add, Concatenate
'''

'\nfrom keras import optimizers\nfrom keras.models import Model, Sequential\nfrom keras.callbacks import EarlyStopping, ModelCheckpoint\nfrom keras.layers import Input, Conv1D, Dense, Activation, Dropout, Flatten, Multiply, Add, Concatenate\n'

In [0]:
import tensorflow as tf
from kerastuner.tuners.bayesian import BayesianOptimization

In [0]:
os.makedirs('model_checkpoints', exist_ok=True)
os.makedirs('plots', exist_ok=True)

In [0]:
df = po.read_csv('data/2017_energy_5min_noTransform.csv')

In [0]:
scaler = MinMaxScaler()
df['W'] = scaler.fit_transform(df)

In [0]:
class Args:
  def __init__(self):
    self.val_start = 12*24*250
    self.test_start = 12*24*300
    self.train_seq_len = 12*24*7
    self.val_seq_len = 12*24
    
    self.batch_size = 64
    self.num_epochs = 10
    self.save_folder = 'tune_WaveNet'
    
    #self.conv_num_channels_1 = 128
    #self.conv_num_channels_2 = 256
    #self.kernel_size = 12
    self.dilation_rates = [2**i for i in range(8)]
    self.logs_dir = 'model_checkpoints/' + self.save_folder + '/logs_dir'

In [0]:
args = Args()

In [0]:
os.makedirs('plots/' + args.save_folder, exist_ok=True)
os.makedirs('model_checkpoints/' + args.save_folder, exist_ok=True)
os.makedirs('model_checkpoints/' + args.save_folder + '/logs_dir', exist_ok=True)
os.makedirs('forecasts/full_testing_length/' + args.save_folder, exist_ok=True)
os.makedirs('forecasts/1_day_at_a_time/' + args.save_folder, exist_ok=True)
os.makedirs('forecasts/1_day_at_a_time/' + args.save_folder + '/plots', exist_ok=True)
os.makedirs('forecasts/1_day_at_a_time/' + args.save_folder + '/tabulated_results', exist_ok=True)

In [0]:
train_df = df[: args.val_start]
val_df = df[args.val_start: args.test_start]
test_df = df[args.test_start:]

In [0]:
def make_windows(data, train_seq_len = args.train_seq_len, val_seq_len = args.val_seq_len):
    X = []
    y = []

    for i in tqdm(range(train_seq_len, len(data) - val_seq_len + 1)):    
        X.append(data.values[i - train_seq_len: i].T.tolist())
        y.append(data.values[i: i + val_seq_len].T.tolist())

    X = np.array(X).squeeze().reshape(-1, train_seq_len, 1)
    y = np.array(y).squeeze()

    return X, y

In [15]:
X_train, y_train = make_windows(train_df)
X_val, y_val = make_windows(val_df)

100%|██████████| 12097/12097 [00:01<00:00, 8061.86it/s]


In [0]:
sgd = tf.keras.optimizers.SGD(lr=0.1, decay=1e-2, momentum=0.9, nesterov=True)
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2)

In [0]:
def build_model(hp):

  num_channels_1 = hp.Int('num_channels', min_value=25, max_value=175, step=50)
  num_channels_2 = hp.Int('num_channels', min_value=25, max_value=175, step=50)
  kernel_size = hp.Int('kernel_size', min_value=6, max_value=36, step=6)
  dropout = hp.Choice('dropout', values=[0.1, 0.2, 0.3])


  history_seq = tf.keras.layers.Input(shape=(args.train_seq_len, 1))
  x = history_seq

  skips = []
  for dilation_rate in args.dilation_rates:
      x = tf.keras.layers.Conv1D(num_channels_1, 1, padding='same', activation='relu')(x) 
      
      # filter convolution
      x_f = tf.keras.layers.Conv1D(filters = num_channels_2,
                  kernel_size = kernel_size, 
                  padding='causal',
                  dilation_rate = dilation_rate)(x)
      
      # gating convolution
      x_g = tf.keras.layers.Conv1D(filters = num_channels_2,
                  kernel_size = kernel_size, 
                  padding='causal',
                  dilation_rate = dilation_rate)(x)
      
      # multiply filter and gating branches
      z = tf.keras.layers.Multiply()([tf.keras.layers.Activation('tanh')(x_f),
                      tf.keras.layers.Activation('sigmoid')(x_g)])
      
      # postprocessing - equivalent to time-distributed dense
      z = tf.keras.layers.Conv1D(num_channels_1, 1, padding='same', activation='relu')(z)
      
      # residual connection
      x = tf.keras.layers.Add()([x, z])    
      
      # collect skip connections
      skips.append(z)

  # add all skip connection outputs 
  out = tf.keras.layers.Activation('relu')(tf.keras.layers.Add()(skips))

  # final time-distributed dense layers 
  out = tf.keras.layers.Conv1D(num_channels_1, 1, padding='same')(out)
  out = tf.keras.layers.Activation('relu')(out)
  out = tf.keras.layers.Dropout(dropout)(out)
  out = tf.keras.layers.Conv1D(1, 1, padding='same')(out)

  out = tf.keras.layers.Flatten()(out)
  out = tf.keras.layers.Dense(args.val_seq_len)(out)

  model = tf.keras.Model(history_seq, out)

  model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['mean_squared_error'])

  return model

In [0]:
tuner = BayesianOptimization(
    build_model,
    objective='mean_squared_error',
    max_trials=20,
    executions_per_trial=2,
    directory=args.logs_dir,
    project_name=args.save_folder,
    overwrite = True)

In [19]:
tuner.search_space_summary()

In [0]:
tuner.search(X_train, y_train, batch_size = args.batch_size, epochs = args.num_epochs, validation_data = (X_val, y_val), callbacks = [earlystop], verbose=True)

Epoch 1/10
 253/1090 [=====>........................] - ETA: 12:00 - loss: 0.0270 - mean_squared_error: 0.0270

In [0]:
tuner.results_summary()

In [0]:
best_epoch = 10#np.argmin(np.array(history.history['val_loss']))+1
model.load_weights('model_checkpoints/' + args.save_folder + '/model_{:02d}.h5'.format(best_epoch))
best_epoch

10

In [0]:
plot_df = po.DataFrame.from_dict({'train_loss':history.history['loss'], 'val_loss':history.history['val_loss']})
plot_df.plot(logy=True, figsize=(10,10), fontsize=12)
plt.xlabel('epoch', fontsize=12)
plt.ylabel('loss', fontsize=12)
plt.savefig('plots/'  + args.save_folder + '/train_val_loss.png')
plt.show()

In [0]:
X_test, y_test = make_windows(test_df)

100%|██████████| 12097/12097 [00:01<00:00, 9482.01it/s]


In [0]:
y_pred = model.predict(X_test)

In [0]:
plt.plot(y_test[0], color = 'blue')
plt.plot(y_pred[0], color = 'red')
plt.savefig('plots/' + args.save_folder + '/test_weekend_pred.png')

In [0]:
plt.plot(y_test[12*24*2], color = 'blue')
plt.plot(y_pred[12*24*2], color = 'red')
plt.savefig('plots/'  + args.save_folder + '/test_weekday_pred.png')

In [0]:
if not os.path.exists('results.csv'):
  results = po.DataFrame(columns = ['model', 'MAE', 'MSE', 'R2'])  
else:
  results = po.read_csv('results.csv')

In [0]:
errors = {'model': args.save_folder,
          'MAE': mean_absolute_error(y_true=y_test, y_pred=y_pred),
          'MSE': mean_squared_error(y_true=y_test, y_pred=y_pred),
          'R2': r2_score(y_true=y_test, y_pred=y_pred)
          }
errors = po.Series(errors)
results = results.append(errors, ignore_index=True)

In [0]:
results

In [0]:
results.to_csv('results.csv', index=False)

In [0]:
with open('model_checkpoints/' + args.save_folder + '/params.json', 'w') as f:
  f.write(json.dumps(args.__dict__))
f.close()

In [0]:
if not os.path.exists('forecasts/full_testing_length/results.csv'):
  results = po.DataFrame(columns = ['model', 'MAE', 'MSE', 'R2', 'MAPE', 'RMSE'])  
else:
  results = po.read_csv('forecasts/full_testing_length/results.csv')

In [0]:
errors = {'model': args.save_folder,
          'MAE': mean_absolute_error(y_true=y_test, y_pred=y_pred),
          'MAPE': mean_absolute_error(y_true=y_test, y_pred=y_pred)*100,
          'MSE': mean_squared_error(y_true=y_test, y_pred=y_pred),
          'RMSE': mean_squared_error(y_true=y_test, y_pred=y_pred, squared=False),
          'R2': r2_score(y_true=y_test, y_pred=y_pred)          
          }
errors = po.Series(errors)
results = results.append(errors, ignore_index=True)

In [0]:
results

,model,MAE,MSE,R2,MAPE,RMSE
0,Dilated_CNN,0.037945,0.002629,0.732376,3.794527,0.051273
1,vanillaWaveNet_256_channels,0.046997,0.003457,0.648037,4.699682,0.058799
2,convLSTM,0.040161,0.002803,0.714712,4.016132,0.052941
3,bidirectionalLSTM,0.126920,0.020101,-1.046449,12.691955,0.141780
4,WaveNet,0.049359,0.004077,0.585029,4.935853,0.063849


In [0]:
results.to_csv('forecasts/full_testing_length/results.csv', index=False)